In [14]:
#https://www.kaggle.com/adityapachpande/spaceship-titanic-ml-neuralnetworks
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from sklearn.metrics import accuracy_score
import sklearn
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [15]:
train = pd.read_csv('../input/spaceship-titanic/train.csv')
test = pd.read_csv('../input/spaceship-titanic/test.csv')

In [16]:
target = train['Transported']
train = train.drop(['PassengerId','Name','Transported'], axis=1)
test = test.drop(['PassengerId','Name'], axis=1)

cat_columns = train.dtypes[(train.dtypes == "object") == True].index.tolist()
for col in cat_columns:
    if col != 'Cabin':
        train[col] = train[col].fillna(f'None{col}')
train['Cabin'] = train['Cabin'].fillna(train['Cabin'].mode()[0])

num_columns = train.dtypes[(train.dtypes != "object") == True].index.tolist()
for col in num_columns:
    train[col] = train[col].fillna(train[col].median())
    
cat_columns = test.dtypes[(test.dtypes == "object") == True].index.tolist()
for col in cat_columns:
    if col != 'Cabin':
        test[col] = test[col].fillna(f'None{col}')
test['Cabin'] = test['Cabin'].fillna(test['Cabin'].mode()[0])

In [17]:
num_columns = test.dtypes[(test.dtypes != "object") == True].index.tolist()
for col in num_columns:
    test[col] = test[col].fillna(test[col].median())

target[target == False] = 0
target[target == True] = 1

train = pd.concat((train, pd.get_dummies(train[['HomePlanet', 'CryoSleep', 'Destination', 'VIP']])), axis=1)
train = train.drop(['HomePlanet', 'CryoSleep', 'Destination', 'VIP'],axis=1)
test = pd.concat((test, pd.get_dummies(test[['HomePlanet', 'CryoSleep', 'Destination', 'VIP']])), axis=1)
test = test.drop(['HomePlanet', 'CryoSleep', 'Destination', 'VIP'],axis=1)

train = train.drop(train['Cabin'][train['Cabin'] == 'NoneCabin'].index, axis=0)
test = test.drop(test['Cabin'][test['Cabin'] == 'NoneCabin'].index, axis=0)

In [18]:
first, sec, thrd = [], [], []
mem = []
for i in range(train['Cabin'].shape[0]):
    mem = train['Cabin'].iloc[i].split('/')
    first.append(mem[0])
    sec.append(mem[1])
    thrd.append(mem[2])
first = pd.Series(first)
first.index = train['Cabin'].index
sec = pd.Series(sec)
sec.index = train['Cabin'].index
thrd = pd.Series(thrd) 
thrd.index = train['Cabin'].index
train['CabinClass'] = first
train['CabinNumber'] = sec
train['CabinSeat'] = thrd

first, sec, thrd = [], [], []
mem = []
for i in range(test['Cabin'].shape[0]):
    mem = test['Cabin'].iloc[i].split('/')
    first.append(mem[0])
    sec.append(mem[1])
    thrd.append(mem[2])
first = pd.Series(first)
first.index = test['Cabin'].index
sec = pd.Series(sec)
sec.index = test['Cabin'].index
thrd = pd.Series(thrd) 
thrd.index = test['Cabin'].index
test['CabinClass'] = first
test['CabinNumber'] = sec
test['CabinSeat'] = thrd

train = train.drop('Cabin',axis=1)
test = test.drop('Cabin',axis=1)

In [19]:
test.dtypes[(test.dtypes == "object") == True].index.tolist()
test.dtypes[(test.dtypes != "object") == True].index.tolist() == train.dtypes[(train.dtypes != "object") == True].index.tolist()

train = pd.concat((train, pd.get_dummies(train['CabinClass'])), axis=1)
train = pd.concat((train, pd.get_dummies(train['CabinSeat'])), axis=1)
train = train.drop(['CabinClass','CabinSeat'],axis=1)

test = pd.concat((test, pd.get_dummies(test['CabinClass'])), axis=1)
test = pd.concat((test, pd.get_dummies(test['CabinSeat'])), axis=1)
test = test.drop(['CabinClass','CabinSeat'],axis=1)

In [20]:
train.columns
test.columns

Index(['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'HomePlanet_Earth', 'HomePlanet_Europa', 'HomePlanet_Mars',
       'HomePlanet_NoneHomePlanet', 'CryoSleep_False', 'CryoSleep_True',
       'CryoSleep_NoneCryoSleep', 'Destination_55 Cancri e',
       'Destination_NoneDestination', 'Destination_PSO J318.5-22',
       'Destination_TRAPPIST-1e', 'VIP_False', 'VIP_True', 'VIP_NoneVIP',
       'CabinNumber', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'T', 'P', 'S'],
      dtype='object')

In [21]:
train = train.astype('float64')
test = test.astype('float64')
target = target.astype('float64')

lst_for_scale = []
for col in train.columns:
    if len(train[col].value_counts()) != 2:
        lst_for_scale.append(col)



scaler = StandardScaler()
scaled_train = scaler.fit_transform(train[lst_for_scale])
scaled_test = scaler.transform(test[lst_for_scale])
scaled_train = pd.DataFrame(scaled_train)

for i in range(0,7):
    scaled_train = scaled_train.rename(columns={i:scaler.feature_names_in_[i]})
    
copy_train = train.drop(scaler.feature_names_in_,axis=1).copy()
train_final = pd.concat((scaled_train, copy_train), axis=1)
scaled_test = pd.DataFrame(scaled_test)

for i in range(0,7):
    scaled_test = scaled_test.rename(columns={i:scaler.feature_names_in_[i]})

copy_test = test.drop(scaler.feature_names_in_,axis=1).copy()
test_final = pd.concat((scaled_test, copy_test), axis=1)

train_final.Age.describe()

count    8.693000e+03
mean    -2.125171e-17
std      1.000058e+00
min     -2.007610e+00
25%     -6.129662e-01
50%     -1.248409e-01
75%      5.724810e-01
max      3.501233e+00
Name: Age, dtype: float64

In [22]:
kfold= KFold(n_splits=10,random_state=42,shuffle=True, ) #kfold cross validation, 10-subsets
X_train, X_test, y_train, y_test = train_test_split(train_final, target, test_size=0.15,random_state=17, )
X_train = torch.tensor(X_train.to_numpy()).float(); X_train.shape
y_train = torch.tensor(y_train.to_numpy()).view(-1,1).float(); y_train.shape
X_test = torch.tensor(X_test.to_numpy()).float(); X_test.shape
print(X_test)
y_test = torch.tensor(y_test.to_numpy()).view(-1,1).float(); y_test.shape

model = nn.Sequential(nn.Linear(31,992),
                      nn.ReLU(),
                      nn.Linear(992,496),
                      nn.ReLU(),
                      nn.Linear(496,248),
                      nn.ReLU(),
                      nn.Linear(248,124),
                      nn.ReLU(),
                      nn.Linear(124,1),
                      nn.Sigmoid())
                      

criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.95)
accuracy_score(y_test.cpu(), (model(X_test).cpu().data.numpy() > 0.5))
accuracy_min = 0

for e in range(150):    
    model.train()
    counter = 0
    for i in range(0,116):
        if counter != 7360:
            counter += 64
            X_batch = X_train[counter-64:counter]
            y_batch = y_train[counter-64:counter].view(-1,1)
            optimizer.zero_grad()
            output = model(X_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()
        elif counter == 7360:
            counter += 29
            X_batch = X_train[counter-29:counter]
            y_batch = y_train[counter-29:counter].view(-1,1)
            optimizer.zero_grad()
            output = model(X_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()
    model.eval()
    loss = criterion(model(X_test), y_test)
    accuracy_ = accuracy_score(y_test.cpu(), (model(X_test).cpu().data.numpy() > 0.5))
    print(f"training loss of {e}th epoch number: {loss}, accuracy: {accuracy_}")
    if accuracy_ >= accuracy_min:
        print('Accuracy metrics increased ({:.6f} --> {:.6f}).  Saving model ...'.format(accuracy_min,accuracy_))
        torch.save(model.state_dict(), '/kaggle/working/model_space.pt')
        accuracy_min = accuracy_
model.load_state_dict(torch.load('/kaggle/working/model_space.pt'))
accuracy_score(y_test.cpu(), (model(X_test).cpu().data.numpy() > 0.5))

tensor([[ 1.4093, -0.3331, -0.2810,  ...,  0.0000,  0.0000,  1.0000],
        [-0.4735, -0.3331, -0.2704,  ...,  0.0000,  0.0000,  1.0000],
        [-0.9616, -0.3331, -0.2810,  ...,  0.0000,  0.0000,  1.0000],
        ...,
        [-0.1248, -0.3331,  0.2573,  ...,  0.0000,  0.0000,  1.0000],
        [-0.4038, -0.2892, -0.2597,  ...,  0.0000,  0.0000,  1.0000],
        [ 0.0146, -0.3331, -0.2810,  ...,  0.0000,  0.0000,  1.0000]])
training loss of 0th epoch number: 0.6894974708557129, accuracy: 0.6104294478527608
Accuracy metrics increased (0.000000 --> 0.610429).  Saving model ...
training loss of 1th epoch number: 0.6827245354652405, accuracy: 0.7615030674846626
Accuracy metrics increased (0.610429 --> 0.761503).  Saving model ...
training loss of 2th epoch number: 0.6683590412139893, accuracy: 0.7615030674846626
Accuracy metrics increased (0.761503 --> 0.761503).  Saving model ...
training loss of 3th epoch number: 0.6315164566040039, accuracy: 0.7684049079754601
Accuracy metrics inc

0.8174846625766872

In [23]:
predictions = torch.tensor(test_final.to_numpy()).float(); predictions.shape
predictions = (model(predictions).cpu().detach().numpy() > 0.5)
predictions.shape

(4277, 1)

In [24]:
predictions

array([[ True],
       [False],
       [ True],
       ...,
       [ True],
       [ True],
       [ True]])